In [1]:
from pickle import load, dump
import pandas as pd

In [2]:
with open('data/df.pkl', 'rb') as f:
    df = load(f)

In [3]:
df.head()

,source_url,paragraphs,paragraphs_sent_tok,mean_of_embeddings_vec,sum_of_embeddings_vec
0,https://kumparan.com/ananda-wardhiati-teresia/...,"[[[Ketua, MPR, Zulkifli, Hasan, menyesalkan, k...",[Ketua MPR Zulkifli Hasan menyesalkan kisruh y...,tensor(0.0001),tensor(0.0790)
1,http://www.suara.com/entertainment/2017/09/06/...,"[[[Suara.com, -, Cerita, sekuel, terbaru, Jame...",[Suara.com- Cerita sekuel terbaru James Bond b...,tensor(0.0001),tensor(0.1129)
2,http://www.suara.com/bisnis/2017/03/13/201620/...,"[[[Menteri, Pertanian, Andi, Amran, Sulaiman, ...",[Menteri Pertanian Andi Amran Sulaiman mengata...,tensor(0.0004),tensor(0.3413)
3,http://poskotanews.com/2017/12/23/kalahkan-rea...,"[[[SPANYOL, –, Barcelona, berhasil, memboyong,...",[SPANYOL – Barcelona berhasil memboyong kemena...,tensor(-0.0003),tensor(-0.2613)
4,http://www.antaranews.com/berita/659644/jayawi...,"[[[Wamena, (, ANTARA, News, ), -, Pemerintah, ...",[Wamena( ANTARA News)- Pemerintah Kabupaten Ja...,tensor(3.9441e-05),tensor(0.0303)


In [4]:
df.shape

(93870, 5)

In [10]:
from get_embeddings import mean_pooling_bert
from similar_documents_retrieval import get_similar_docs_by_cos_sims
from detailed_analysis import get_plagiarised_pairs
from datetime import datetime

with open('../data/indosum/df.pkl', 'rb') as f:
    corpus = load(f)

def transform_suspicious_text(sus_text):
    print('[INFO] Transforming suspicious text into IndoBERT embeddings...')
    embeddings = mean_pooling_bert(text=sus_text)
    return embeddings

def get_n_similar_documents(sus_text, n=5):
    embeddings = transform_suspicious_text(sus_text=sus_text)
    print(f'[INFO] Getting {n} similar documents by cosine similarity...')
    similar_docs = get_similar_docs_by_cos_sims(sus_doc_embeddings=embeddings, n=n)
    return similar_docs

def detailed_plagiarism(sus_text, n=5, threshold=.9):
    similar_sources = get_n_similar_documents(sus_text=sus_text, n=n)
    similar_source_docs = {}
    suspected_plagiarism_cases = {}
    print('[INFO] Getting source documents..')
    for src_idx in similar_sources:
        similar_source_docs[src_idx] = corpus['paragraphs_sent_tok'][src_idx]
        print(f'[INFO] Source document {src_idx} appended')

    print('[INFO] Performing detailed analysis...')
    for doc_idx, doc in similar_source_docs.items():
        # print(doc)
        suspected_plagiarism_cases[doc_idx] = get_plagiarised_pairs(src_doc=doc, sus_doc=sus_text, threshold=threshold)
        print(f'[INFO] Source document {doc_idx} checked')

    return suspected_plagiarism_cases

In [11]:
# test
document = '''Versi web Google Calendar memiliki sesuatu yang baru untuk ditawarkan. Penampilannya telah sepenuhnya diperbarui, mengadopsi gaya desain modern yang sebelumnya tersedia pada aplikasi Google Calendar di perangkat mobile. Secara keseluruhan, versi Calendar baru terlihat lebih rapi dan lebih mudah untuk diarahkan. Layout responsif yang digunakan berarti penampilannya akan disesuaikan secara otomatis dengan ukuran layar dan jendela browser. Bukan hanya berubah secara visual, versi Calendar baru juga membawa beberapa fitur yang berguna. Salah satunya adalah kemampuan untuk memformat dan menambahkan tautan pada undangan, sehingga pengguna dapat membuat jadwal yang lebih detail dan memberikan akses kepada materi yang dibutuhkan oleh peserta lain sebelum rapat dimulai. Calendar sekarang juga membuat lebih mudah untuk melihat dan mengelola beberapa kalender dari akun yang berbeda dalam tampilan harian. Fitur ini tentunya akan sangat membantu pekerja yang harus menjadwalkan rapat untuk anggota tim mereka. Terakhir, saat membuat jadwal rapat baru, pengguna dapat menambahkan informasi yang lebih detail tentang ruang rapat, termasuk lokasi, kapasitas, peralatan audio dan video, dan informasi tentang aksesibilitas bagi peserta yang berkedudukan kursi roda. Bagi yang menggunakan Calendar untuk akun pribadi mereka, mereka bisa mengaktifkan tampilan baru dengan mengklik tombol "Use new Calendar" di ujung kanan atas. Administrator G Suite juga dapat mengaktifkan tampilan baru Calendar mulai hari ini. Sumber: Google. DailySocial.id adalah portal berita startup dan inovasi teknologi. Kamu bisa menjadi anggota komunitas startup dan inovasi DailySocial.id, mengunduh laporan riset dan statistik seputar teknologi secara gratis, dan mengikuti berita startup dan gadget terbaru di Indonesia.'''
# sentences = sent_tokenize(document)
# print(sentences)
start = datetime.now()
plagiarism_cases = detailed_plagiarism(sus_text=document, n=5, threshold=.9)
print(plagiarism_cases)
end = datetime.now()
total_time = end-start
print(f"total time: {total_time}")

[INFO] Transforming suspicious text into IndoBERT embeddings...
[INFO] Getting 5 similar documents by cosine similarity...
[INFO] Getting source documents..
[INFO] Source document 5 appended
[INFO] Source document 22493 appended
[INFO] Source document 41106 appended
[INFO] Source document 60767 appended
[INFO] Source document 75240 appended
[INFO] Performing detailed analysis...
[INFO] Length of source: 14
[INFO] Length of sus: 14
[INFO] Source document 5 checked
[INFO] Length of source: 14
[INFO] Length of sus: 14
[INFO] Source document 22493 checked
[INFO] Length of source: 14
[INFO] Length of sus: 14
[INFO] Source document 41106 checked
[INFO] Length of source: 14
[INFO] Length of sus: 14
[INFO] Source document 60767 checked
[INFO] Length of source: 14
[INFO] Length of sus: 14
[INFO] Source document 75240 checked
{5: [('Tampilannya kini telah dirombak habis, mengadopsi gaya desain modern yang sebelumnya sudah tersedia pada aplikasi Google Calendar di perangkat mobile.', 'Penampilann